<a href="https://colab.research.google.com/github/neha-duggirala/Yoga-Instructor/blob/master/Yoga_Instructor_ALL1_8_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls "/content/drive/My Drive/datasets/Yoga DataSet/"

'Demo for BFDI.mp4'  'demo yoga.mp4'   numpy_data/   to_be_Sorted/
'Demo Images'/        models/          test.jpg      yogasteps.docx


In [ ]:
from tqdm import tqdm
import numpy as np
import cv2
import os,time
from random import shuffle
import matplotlib.pyplot as plt
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.callbacks import TensorBoard
print ("Imported!",tf.__version__)

Imported! 2.1.0


In [ ]:
def convertRGB(numpy_data):
    data = []
    for gray in numpy_data:
          colored= cv2.cvtColor(gray[0],cv2.COLOR_GRAY2RGB)
          data.append([colored,gray[1]])
    data = np.asarray(data)
    return data


def XYsplit(data):
    X= np.asarray([images[0] for images in data])
    y= np.asarray([images[1] for images in data])
    print(X.shape,y.shape,X[0].shape)
    return X,y

def numpy_attach(src,parts):
  #need to change this name
    name = src+"(90, 90)-Train-2397_data"
    data = np.load(name+"0.npy",allow_pickle=True)
    # data/=255.0
    for i in range(1,parts):
        n=name+str(i)+".npy"
        new_data = np.load(n,allow_pickle=True)
        # try:
        #   new_data/=255.0
        # except:
          # print("None Error")
        print(n,len(data),(new_data).shape)
        data = np.concatenate((data,new_data))
      
    return data

def normalize(array,parts):
  start = 0
  end= start+(len(array)//parts)
  print(end)
  data=np.array(array[start:start+end])
  for i in range(1,parts):
    end= start+(len(array)//parts)
    array[start:start+end]=array[start:start+end]/255.0
  return array

In [ ]:
ls "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail4"

'(224, 224)-Test-598_data.npy'    '(300, 300)-Test-1044_data.npy'
'(224, 224)-Train-4109_data.npy'  '(300, 300)-Train-3420_data.npy'


In [ ]:


test_path = "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail4/(300, 300)-Test-1044_data.npy"
train_path = "/content/drive/My Drive/datasets/Yoga DataSet/numpy_data/ALL1-8/Trail4/(300, 300)-Train-3420_data.npy"


In [ ]:

training_data=np.load(train_path,allow_pickle=True)
test_data=np.load(test_path,allow_pickle=True)


In [ ]:
training_data=convertRGB(training_data)
test_data=convertRGB(test_data)


In [ ]:
trainX,trainY = XYsplit(training_data)
testX,testY = XYsplit(test_data)



(3420, 300, 300, 3) (3420,) (300, 300, 3)
(1044, 300, 300, 3) (1044,) (300, 300, 3)


In [ ]:
print("Normalizing")
trainX =trainX/255.0
testX = testX /255.0

Normalizing


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!unzip keras-cyclical-learning-rates.zip

In [ ]:
%cd keras-cyclical-learning-rates/pyimagesearch/

In [ ]:
import sys,tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D , AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input,BatchNormalization

In [ ]:
# mobileNet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", 
#                                                            include_top=False,
#                                                       input_tensor=Input(shape=trainX[0].shape),
#                                                       alpha =0.75                                                )

# for layers in mobileNet_v2.layers[1:]:
#   mobileNet_v2.trainable=False
# # mobileNet_v2.summary()

In [ ]:
# mobileNet = tf.keras.applications.mobilenet.MobileNet(weights = "imagenet", 
#                                                            include_top=False,
#                                                       input_tensor=Input(shape=trainX[0].shape),
#                                                       # alpha = 
#                                                       )
# for layers in mobileNet.layers[1:]:
#   mobileNet.trainable=False
# # mobileNet.summary()

In [ ]:
# inception_resnet = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights="imagenet",
# 		# classes=8,
#     include_top=False,																					                     
# 	input_tensor=Input(shape=trainX[0].shape))
# for layers in inception_resnet.layers[1:]:
#   inception_resnet.trainable=False
# # inception_resnet.summary()

In [ ]:
# inception = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet",
# 		# classes=8,
#     include_top=False,																					                     
# 	input_tensor=Input(shape=trainX[0].shape))
# for layers in inception.layers[1:]:
#   inception.trainable=False
# inception.summary()

In [ ]:
# resnet = tf.keras.applications.resnet.ResNet152(weights="imagenet",
# 		# classes=8,
#     include_top=False,																					                     
# 	input_tensor=Input(shape=trainX[0].shape))
# for layers in resnet.layers[1:]:
#   resnet.trainable=False
# resnet.summary()

In [ ]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=trainX[0].shape))
for layers in vgg16.layers[1:]:
  layers.trainable=False

In [ ]:
headModel = vgg16.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1028,activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = BatchNormalization(axis=1)(headModel)
headModel = Dense(685, activation="relu")(headModel)
headModel = Dropout(0.45)(headModel)
headModel = BatchNormalization(axis=1)(headModel)
headModel = Dense(300, activation="relu")(headModel)
# headModel = Dropout(0.45)(headModel)
headModel = BatchNormalization(axis=1)(headModel)
headModel = Dense(8, activation="softmax")(headModel)
 

model = Model(inputs=vgg16.input, outputs=headModel)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

In [ ]:
from keras.callbacks import *
from clr_callback import *
from keras.optimizers import Adam

# You are using the triangular learning rate policy and
#  base_lr (initial learning rate which is the lower boundary in the cycle) is 0.1
clr_triangular = CyclicLR(mode='triangular')
model.compile(optimizer=Adam(0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
history = model.fit(trainX, trainY, 
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    # callbacks=[es],
                    validation_split=0.15
                    )

Train on 2907 samples, validate on 513 samples
Epoch 1/20
2907/2907 [==============================] - 194s 67ms/sample - loss: 2.3840 - accuracy: 0.2250 - val_loss: 1.9236 - val_accuracy: 0.3762
Epoch 2/20
2907/2907 [==============================] - 188s 65ms/sample - loss: 1.7284 - accuracy: 0.4107 - val_loss: 1.6278 - val_accuracy: 0.4016
Epoch 3/20
2907/2907 [==============================] - 188s 65ms/sample - loss: 1.3800 - accuracy: 0.5236 - val_loss: 1.3187 - val_accuracy: 0.5166
Epoch 4/20
2907/2907 [==============================] - 189s 65ms/sample - loss: 1.0881 - accuracy: 0.6147 - val_loss: 1.1015 - val_accuracy: 0.6062
Epoch 5/20
2907/2907 [==============================] - 189s 65ms/sample - loss: 0.8665 - accuracy: 0.6894 - val_loss: 0.9265 - val_accuracy: 0.6959
Epoch 6/20
2907/2907 [==============================] - 191s 66ms/sample - loss: 0.7322 - accuracy: 0.7393 - val_loss: 0.9448 - val_accuracy: 0.6550
Epoch 7/20
2907/2907 [==============================] - 191

In [ ]:
model.evaluate(testX,np.array(testY))

1044/1044 [==============================] - 47s 45ms/sample - loss: 2.6357 - acc: 0.5651


[2.635702184333655, 0.5651341]

In [ ]:
def plot_graphs(history):
  plt.plot(history.history["val_acc"])
  plt.plot(history.history["val_loss"])
  plt.plot(history.history["acc"])
  plt.plot(history.history["loss"])
  plt.xlabel("Epochs")
  plt.ylabel("accuracy")
  plt.legend(["Validation Accuracy","Validation loss","Accuracy","Loss"])
  plt.show()
plot_graphs(history)

In [ ]:
model.save("/content/drive/My Drive/datasets/Yoga DataSet/models/ALL1-8/Trail4/inception_resnet-(300,300,3)E.h5")

In [ ]:
history = model.fit(trainX, trainY, 
                    batch_size=128,
                    epochs=5,
                    verbose=1,
                    # callbacks=[es],
                    validation_split=0.2
                    )

Train on 1908 samples, validate on 478 samples
Epoch 1/5
1908/1908 [==============================] - 34s 18ms/sample - loss: 0.0327 - acc: 0.9916 - val_loss: 0.8701 - val_acc: 0.7699
Epoch 2/5
1908/1908 [==============================] - 34s 18ms/sample - loss: 0.0386 - acc: 0.9885 - val_loss: 0.9040 - val_acc: 0.7594
Epoch 3/5
1908/1908 [==============================] - 35s 18ms/sample - loss: 0.0331 - acc: 0.9885 - val_loss: 1.0495 - val_acc: 0.7301
Epoch 4/5
1908/1908 [==============================] - 34s 18ms/sample - loss: 0.0294 - acc: 0.9874 - val_loss: 1.1522 - val_acc: 0.7155
Epoch 5/5
1908/1908 [==============================] - 34s 18ms/sample - loss: 0.0276 - acc: 0.9906 - val_loss: 1.0954 - val_acc: 0.7322


In [ ]:
model.evaluate(testX,np.array(testY))

598/598 [==============================] - 6s 10ms/sample - loss: 1.2258 - acc: 0.7007


[1.2258075455758086, 0.7006689]

In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file( "/content/drive/My Drive/datasets/Yoga DataSet/models/ALL1-8/Trail1/VGG16-(300,300,3)D.h5" ) 
model = converter.convert()

file = open( '/content/drive/My Drive/datasets/Yoga DataSet/models/ALL1-8/Trail1/VGG16-(300,300,3)D.tflite' , 'wb' ) 
file.write( model )
